In [11]:
#Utility functions for ensuring validity of user input, specifically whether the bounding box exists on earth or not
def check_longitude_validity(min_longitude, max_longitude):
    if min_longitude > max_longitude:
        raise Exception("minimum longitude must be less than maximum longitude")
    if min_longitude < -180.0 or min_longitude > 180.0:
        raise Exception("minimum longitude is not a valid value for longitude")
    if max_longitude < -180.0 or max_longitude > 180.0:
        raise Exception("maximum longitude is not a valid value for longitude")


def check_latitude_validity(min_latitude, max_latitude):
    if min_latitude > max_latitude:
        raise Exception("minimum latitude must be less than maximum latitude")
    if min_latitude < -90.0 or min_latitude > 90.0:
        raise Exception("minimum latitude is not a valid value for latitude")
    if max_latitude < -90.0 or max_latitude > 90.0:
        raise Exception("maximum latitude is not a valid value for latitude")


def check_bounding_box_validity(lon_lat_min, lon_lat_max):
    check_longitude_validity(min_longitude=lon_lat_min[0], max_longitude=lon_lat_max[0])
    check_latitude_validity(min_latitude=lon_lat_min[1], max_latitude=lon_lat_max[1])
    


In [22]:
#class for organizing all user provided options for rtc-gamma and composite process.
#In the original script this is done through command line options

class compositeOptions():
    def init(self):
        return None
    def get_options(self):
        self.earthdata_username=input("provide your earthdata username: ")
        self.earthdata_password=input("provide your earthdata password: ")
        self.start_date=input("provide start date for granule search in the form of yyyy-mm-dd or leave blank: ")
        self.end_date=input("provide end date for granule search in the form of yyyy-mm-dd or leave blank: ")
        self.lon_lat_minimum=input("provide a coordinate to define lower left corner of bounding box in the form 'min_lon, min_lat': ")
        self.lon_lat_maximum=input("provide a coordinate to define upper right corner of bounding box in the form 'max_lon, max_lat': ")
        self.granule_count=input("provide the maximum number of granules for composite process: ")
        self.polarization=input("provide desired polarization of granules [vv, vh, vv+vh]") 
        
        self.parse_bounding_box()
    
    def parse_bounding_box(self):
        self.lon_lat_minimum = self.lon_lat_minimum.split(" ")
        self.lon_lat_minimum[0] = float(self.lon_lat_minimum[0])
        self.lon_lat_minimum[1] = float(self.lon_lat_minimum[1])
        
        self.lon_lat_maximum = self.lon_lat_maximum.split(" ")
        self.lon_lat_maximum[0] = float(self.lon_lat_maximum[0])
        self.lon_lat_maximum[1] = float(self.lon_lat_maximum[1])
        
        check_bounding_box_validity(self.lon_lat_minimum, self.lon_lat_maximum)

In [24]:
#Prompt user to provide input and create object holding that input for duration of processing 
composite_options=compositeOptions()
composite_options.get_options()

provide your earthdata username: rr
provide your earthdata password: r
provide start date for granule search in the form of yyyy-mm-dd or 'None': 
provide a coordinate to define lower left corner of bounding box in the form 'min_lon, min_lat': -120.0 68.4
provide a coordinate to define upper right corner of bounding box in the form 'max_lon, max_lat': -118.0 69.0
provide the maximum number of granules for composite process: 5


In [9]:
#Main utility functions for buidling earthdata search url

def get_fetch_url_bbox(composite_options):
    min_bbox_bound = composite_options.lon_lat_minimum
    max_bbox_bound = composite_options.lon_lat_maximum

    fetch_url_bbox = "bbox={min_lon},{min_lat},{max_lon},{max_lat}&".format(
        min_lon=min_bbox_bound[0],
        min_lat=min_bbox_bound[1],
        max_lon=max_bbox_bound[0],
        max_lat=max_bbox_bound[1]
        )
    return fetch_url_bbox


def get_fetch_url_date_range(composite_options):
    fetch_url_date_range = "start={start_date}T00:00:00UTC&end={end_date}T00:00:00&".format(
        start_date=composite_options.start_date if composite_options.start_date is not None else "2014-06-15",
        end_date=composite_options.end_date if composite_options.end_date is not None else str(date.today())
        )
    return fetch_url_date_range


def get_fetch_url_polarization(composite_options):
    fetch_url_polarization = "polarization={polarization}".format(
        polarization=composite_options.polarization.upper()
    ).replace("+", "%2B")
    return fetch_url_polarization


def get_granule_names(granule_list):
    granule_names = []
    for granule in granule_list:
        name = granule["granuleName"]
        logger.info("Acquired granule {name}".format(name=name))
        granule_names.append(name)

    return granule_names

hadimarchi


In [ ]:
# Search url for granules to process with rtc gamma

def build_search_url(composite_options):
    base_url = "https://api.daac.asf.alaska.edu/services/search/param?"

    fetch_url_bbox = get_fetch_url_bbox(composite_options)

    fetch_url_date_range = get_fetch_url_date_range(composite_options)

    fetch_url_polarization = get_fetch_url_polarization(composite_options)

    fetch_url = "{base_url}{bbox}{date_range}{polarization}&platform=S1&processingLevel=GRD_HD&output=JSON".format(
        base_url=base_url,
        bbox=fetch_url_bbox,
        date_range=fetch_url_date_range,
        polarization=fetch_url_polarization
        )
    logger.debug("Fetch url is {fetch_url}".format(fetch_url=fetch_url))
    return fetch_url

In [ ]:
def fetch_granules(composite_options):
    search_url = build_search_url()
    granules = requests.get(search_url).json()[0] 
    # Somewhat confusing format but .json() call creates a list where the first element
    # is a dictionary containing a variety of information about said granules
    granule_names = get_granule_names(granules)

    if granule_names == []:
        raise Exception("No granules found")
    logger.info("Found {granule_count} granules".format(granule_count=len(granule_names)))
    return granule_names